In [56]:
import numpy as np
from tqdm import tqdm_notebook as tqdm

import numpy as np

# Base code : https://yamalab.tistory.com/92
class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose


    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)]) # rating의 평균이라고 생각하면 된다.

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):
            # rating이 존재하는 index를 기준으로 training
            xi, yi = self._R.nonzero()
            for i, j in zip(xi, yi):
                self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        # predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        return np.sqrt(cost/len(xi))


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) + (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) + (self._reg_param * self._Q[j, :])
        return dp, dq


    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction
        print(error)

        # update biases
        self._b_P[i] += self._learning_rate * (error + self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error + self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)


    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)
        #각각의 차원 1 + (7,1) + (5,) + (7*3)*(3*5)


# run example
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5)
    R = np.array([
        [1, 0, 0, 1, 3],
        [2, 0, 3, 1, 1],
        [1, 2, 0, 5, 0],
        [1, 0, 0, 4, 4],
        [2, 1, 5, 4, 0],
        [5, 1, 5, 4, 0],
        [0, 0, 0, 1, 0],
    ])

    # P, Q is (7 X k), (k X 5) matrix

In [57]:
%%time
factorizer = MatrixFactorization(R, k=3, learning_rate=0.01, reg_param=0.01, epochs=100, verbose=True)
factorizer.fit()

-5.282241231162848
-5.038237500172115
0.07284377436680156
1.915263389723191
1.5975737639906729
0.9340233861269183
-1.2643151417535137
-3.3493406660115372
0.850172427618114
0.6278417835473427
-2.5903534163593003
0.587387729594381
2.4503076192383695
-1.9691221903775928
-2.9176496973814685
1.6669504813182856
-0.12057620057540941
3.7604254840791755
-1.242997667596367
2.8405109072632673
2.582197205537631
-1.3891769148722437
-4.359180701313633
-4.4040159099505765
0.13221531895122762
1.5467703850177426
1.3530874546328684
0.6935976262932759
-1.2710306620670235
-2.9398911255153246
0.8323995150853181
0.8072569846319411
-2.3809942130928485
0.6685170609441657
2.3663076181458242
-1.7165452246844342
-2.68004396950402
1.6308488038728473
-0.005786368145328069
3.339607732664947
-1.1796816932414136
2.4392307241780564
2.211276387563706
-1.329210008191322
-3.6927233592060453
-3.9317248416797277
0.18397847297912007
1.288406017889276
1.1622653643718193
0.5240139478066022
-1.2734588006740823
-2.6279112349443

0.4375402198331688
-0.15973201234636214
0.060407980610119205
0.35346577093637244
-0.19930392442237999
1.0007621656299088
-0.16105492026787127
-0.7271438580299421
0.1149542793643521
-0.1341555340416365
-0.14353353596560137
-0.4332457390585993
0.5681305300263766
-0.15942278965070011
0.27791907771680746
0.3007920857859403
-0.5141686950347408
0.1628518504501888
-0.0012396657264286404
0.1227780889572685
-0.7451425503681597
0.03606805482670605
0.42639648965046995
-0.16280894603311147
0.061794927600455796
0.35345568187838605
-0.19836876014193017
0.9840469308244923
-0.15851041027406798
-0.7243406295861092
0.1183376240011631
-0.13143207267243162
-0.14322237997075216
-0.4270705468389091
0.5620071602786414
-0.16326658516889658
0.27993023535053085
0.2956853437173703
-0.5044995069209917
0.16864504348502818
-0.0034127027932022713
0.11832192071147052
-0.7273014941132048
0.03246933705801336
0.41547606256059844
-0.16589899669458053
0.06305745162098819
0.3534710482888048
-0.19742837436261773
0.967509449

In [3]:
%%time
factorizer = MatrixFactorization(R, k=3, learning_rate=0.01, reg_param=0.01, epochs=100, verbose=True)
factorizer.fit()

Iteration: 10 ; cost = 1.3283
Iteration: 20 ; cost = 1.0175
Iteration: 30 ; cost = 0.8671
Iteration: 40 ; cost = 0.7805
Iteration: 50 ; cost = 0.7207
Iteration: 60 ; cost = 0.6711
Iteration: 70 ; cost = 0.6230
Iteration: 80 ; cost = 0.5709
Iteration: 90 ; cost = 0.5124
Iteration: 100 ; cost = 0.4480
CPU times: user 93.7 ms, sys: 23.8 ms, total: 118 ms
Wall time: 97.4 ms


In [4]:
# 예측값
np.round(factorizer.get_complete_matrix(),2)

array([[1.54, 1.47, 2.76, 1.08, 2.75],
       [1.59, 4.51, 3.04, 0.99, 1.16],
       [2.  , 1.52, 4.7 , 4.32, 3.82],
       [1.56, 0.43, 3.98, 3.82, 3.91],
       [1.67, 1.13, 4.88, 4.24, 3.12],
       [3.86, 1.37, 5.05, 4.34, 6.82],
       [1.14, 1.66, 2.92, 1.17, 1.83]])

In [5]:
# Ground Truth
R

array([[1, 0, 0, 1, 3],
       [2, 0, 3, 1, 1],
       [1, 2, 0, 5, 0],
       [1, 0, 0, 4, 4],
       [2, 1, 5, 4, 0],
       [5, 1, 5, 4, 0],
       [0, 0, 0, 1, 0]])

# ALS 알고리즘

In [6]:
import numpy as np
from tqdm import tqdm_notebook as tqdm

# Base code : https://github.com/mickeykedia/Matrix-Factorization-ALS/blob/master/ALS%20Python%20Implementation.py
class AlternatingLeastSquares():
    def __init__(self, R, k, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose


    def fit(self):
        # init latent features
        self._users = np.random.normal(size=(self._num_users, self._k))
        self._items = np.random.normal(size=(self._num_items, self._k))

        # train while epochs
        self._training_process = []
        self._user_error = 0; self._item_error = 0; 
        for epoch in range(self._epochs):
            for i, Ri in enumerate(self._R):
                self._users[i] = self.user_latent(i, Ri)
                self._user_error = self.cost()
                
            for j, Rj in enumerate(self._R.T):
                self._items[j] = self.item_latent(j, Rj)
                self._item_error = self.cost()
                
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """
        xi, yi = self._R.nonzero()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        return np.sqrt(cost/len(xi))


    def user_latent(self, i, Ri):
        """
        :param error: rating - prediction error
        :param i: user index
        :param Ri: Rating of user index i
        :return: convergence value of user latent of i index
        """

        du = np.linalg.solve(np.dot(self._items.T, np.dot(np.diag(Ri), self._items)) + self._reg_param * np.eye(self._k),
                                   np.dot(self._items.T, np.dot(np.diag(Ri), self._R[i].T))).T
        return du

    def item_latent(self, j, Rj):
        """
        :param error: rating - prediction error
        :param j: item index
        :param Rj: Rating of item index j
        :return: convergence value of itemr latent of j index
        """

        di = np.linalg.solve(np.dot(self._users.T, np.dot(np.diag(Rj), self._users)) + self._reg_param * np.eye(self._k),
                                 np.dot(self._users.T, np.dot(np.diag(Rj), self._R[:, j])))
        return di


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._users[i, :].dot(self._items[j, :].T)


    def get_complete_matrix(self):
        """
        :return: complete matrix R^
        """
        return self._users.dot(self._items.T)



# run example
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5)
    R = np.array([
        [1, 0, 0, 1, 3],
        [2, 0, 3, 1, 1],
        [1, 2, 0, 5, 0],
        [1, 0, 0, 4, 4],
        [2, 1, 5, 4, 0],
        [5, 1, 5, 4, 0],
        [0, 0, 0, 1, 0],
    ])

In [7]:
als = AlternatingLeastSquares(R = R, reg_param = 0.01, epochs=100, verbose=True, k=3)
als.fit()


Iteration: 10 ; cost = 0.0249
Iteration: 20 ; cost = 0.0267
Iteration: 30 ; cost = 0.0250
Iteration: 40 ; cost = 0.0232
Iteration: 50 ; cost = 0.0217
Iteration: 60 ; cost = 0.0204
Iteration: 70 ; cost = 0.0194
Iteration: 80 ; cost = 0.0186
Iteration: 90 ; cost = 0.0179
Iteration: 100 ; cost = 0.0172


In [8]:
np.round(als.get_complete_matrix())

array([[ 1.,  2.,  4.,  1.,  3.],
       [ 2.,  1.,  3.,  1.,  1.],
       [ 1.,  2.,  7.,  5.,  9.],
       [ 1., -1.,  2.,  4.,  4.],
       [ 2.,  1.,  5.,  4.,  6.],
       [ 5.,  1.,  5.,  4.,  2.],
       [-0., -1.,  0.,  1.,  1.]])

In [9]:
R

array([[1, 0, 0, 1, 3],
       [2, 0, 3, 1, 1],
       [1, 2, 0, 5, 0],
       [1, 0, 0, 4, 4],
       [2, 1, 5, 4, 0],
       [5, 1, 5, 4, 0],
       [0, 0, 0, 1, 0]])